## Neo4j Operations

In this section, we will demonstrate how to connect to a  Neo4j Graph Database and perform basic CRUD operations. We will use the `GraphDatabase` library.



In [ ]:
pip install neo4j

Setup Connection

In [36]:
from neo4j import GraphDatabase

# Replace with your Neo4j credentials and URL
uri = "bolt://localhost:7687"
username = "neo4j"
password = "daanvi19"

driver = GraphDatabase.driver(uri, auth=(username, password))


Define Query Execution Function

In [38]:
def execute_query(query, parameters=None):
    with driver.session() as session:
        result = session.run(query, parameters)
        return result.data()


Create Nodes and Relationships

In [40]:
# Create departments
execute_query("CREATE (d1:Department {name: 'Engineering'})")
execute_query("CREATE (d2:Department {name: 'Marketing'})")

# Create employees
execute_query("CREATE (e1:Employee {name: 'Alice', title: 'Engineer'})")
execute_query("CREATE (e2:Employee {name: 'Bob', title: 'Marketer'})")

# Establish relationships
execute_query("""
    MATCH (e:Employee {name: 'Alice'}), (d:Department {name: 'Engineering'})
    CREATE (e)-[:WORKS_IN]->(d)
""")

execute_query("""
    MATCH (e:Employee {name: 'Bob'}), (d:Department {name: 'Marketing'})
    CREATE (e)-[:WORKS_IN]->(d)
""")


[]

Retrieve All Employees in a Specific Department

In [22]:
def get_employees_in_department(department_name):
    query = """
    MATCH (e:Employee)-[:WORKS_IN]->(d:Department {name: $department_name})
    RETURN e.name AS employee_name
    """
    result = execute_query(query, parameters={"department_name": department_name})
    return [record['employee_name'] for record in result]

# Retrieve employees in 'Engineering'
employees_in_engineering = get_employees_in_department('Engineering')
print(employees_in_engineering)


['Alice', 'Alice', 'Alice', 'Alice', 'Alice']


In [ ]:
Update an Employee's Department

In [24]:
def update_employee_department(employee_name, new_department_name):
    query = """
    MATCH (e:Employee {name: $employee_name})
    MATCH (d:Department {name: $new_department_name})
    OPTIONAL MATCH (e)-[r:WORKS_IN]->()
    DELETE r
    CREATE (e)-[:WORKS_IN]->(d)
    """
    execute_query(query, parameters={"employee_name": employee_name, "new_department_name": new_department_name})

# Update Alice's department to 'Marketing'
update_employee_department('Alice', 'Marketing')


Remove an Employee from the Graph

In [26]:
def remove_employee(employee_name):
    query = """
    MATCH (e:Employee {name: $employee_name})
    OPTIONAL MATCH (e)-[r:WORKS_IN]->()
    DELETE r, e
    """
    execute_query(query, parameters={"employee_name": employee_name})

# Remove Bob from the graph
remove_employee('Bob')


Visualize or Output Results

In [28]:
import pandas as pd

# Example to visualize employees in 'Engineering'
employees_in_engineering = get_employees_in_department('Engineering')
df = pd.DataFrame(employees_in_engineering, columns=['Employee Name'])
df


,Employee Name


Neo4j Considerations

1.Hierarchical Data: Ideal for modeling and querying complex hierarchical structures.

2.Relationship Efficiency: Optimized for handling and traversing relationships.

3.Performance: Scales well for large graphs with fast traversal capabilities.

4.Scalability: Supports clustering for distributed data storage and processing.

5.Flexible Modeling: Schema-optional, allowing easy adaptation to changing requirements.